# PAM for Analog Source Signals
Markus Gardill, 2019

In [1]:
import numpy as np
# for interactive plots
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot, row, column
from bokeh.resources import INLINE
import matplotlib.pyplot as plt

In [2]:
output_notebook(resources=INLINE)

Loading BokehJS ...

## Symbol Definitions

### For the PAM Signal

The sampling rate $f_s$ as well as sampling period $T_S$

In [3]:
f_s = 1
T_s = 1/f_s 

Define the overall signal/simulation length such that we have $N$ samples

In [4]:
N = 89.45
T = N * T_s

Define simulation time vector

In [5]:
t = np.arange(0,T,T_s)

### For the Source Signal
Since any signal representation on a computer is of discrete-time nature, a continuos-time signal is approximated using a sufficiently large sampling frequency `f_sim`. In addition, an integer multiple `N_sim` between sampling rate of PAM signal and the sampling rate `f_sim` is used to simplify a later expression of the sampling process.

In [6]:
f_s_sim = 100*f_s
T_s_sim = 1/f_s_sim
t_sim = np.arange(0,T,T_s_sim)

## Source Signal

In [7]:
f_source = f_s/8

In [8]:
s = np.sin(2*np.pi*f_source*t_sim)

In [9]:
fig = figure(width = 800, height = 250)
fig.line(x=t_sim, y=s)
fig.xaxis.axis_label = 't'
fig.yaxis.axis_label = 's(t)'
show(fig)

## PAM Signal using Ideal Pulses

The ideal PAM signal is defined as
$$m(t) = \sum\limits_{k=-\infty}^{\infty} s(kT) \delta(t-kT)$$

In the present simulation, this can most easily be acheived by defining an $m(t)$ which is filled with zeros and then copying each `int(f_s_sim/f_s)` value from $s(kT)$ into $m(kT)$.

In [70]:
m = np.zeros(s.shape)
m[1::int(f_s_sim/f_s)] = s[1::int(f_s_sim/f_s)]

In [71]:
fig = figure(width = 800, height = 250)
fig.line(x = t_sim, y = s)
fig.scatter(x = t_sim[m != 0], y = m[m != 0], color='green')
fig.xaxis.axis_label = 't'
fig.yaxis.axis_label = 's(t)'
show(fig)

To illustrate the influence on the signal spectrum, both the FFTs of the source signal $S(f)$ and $M(f)$ are computed and the ffts are shifted by `np.fft.fftshift` such that the zero frequency is in the middle of the resulting array, the largest negative frequency at the beggining and the largest positive frequency at the end of the array.

In [72]:
S = np.fft.fftshift(np.fft.fft(s))
M = np.fft.fftshift(np.fft.fft(m))

The numpy function `np.fft.fftfreq` can be used to compute a vector of frequencies, where the FFT has been evaluated. It is also shifted by `np.fft.fftshift` to center the 0 frequency bin.

In [73]:
freqs = np.fft.fftshift(np.fft.fftfreq(s.shape[0]))

Now the FFTs can be plotted. Since the ideal PAM signal $m(t)$ contains less energy than the source signal (only a fraction of its samples has non-zero values) it can be seen that the energy contained is much smaller.

In [74]:
fig = figure(width = 800, height = 300, title = 'FFT')
fig.line(x = freqs, y = 10*np.log10(np.abs(S)), color = 'blue')
fig.line(x = freqs, y = 10*np.log10(np.abs(M)), color = 'green')
fig.xaxis.axis_label = 'normalized frequency f_n'
fig.yaxis.axis_label = 'S(f_n), M(f_n)'
show(fig)

To focus on the shape of the spectra of source signal and modulated signal, the FFTs are normalized to their respective maximum value before plotting.

In [75]:
fig = figure(width = 800, height = 300, y_range = (-20,5), title = 'Normalized FFT')
fig.line(x = freqs, y = 10*np.log10(np.abs(S)/np.max(np.abs(S))), color = 'blue', legend = 'S(f)')
fig.line(x = freqs, y = 10*np.log10(np.abs(M)/np.max(np.abs(M))), color = 'green', legend = 'M(f)')
fig.xaxis.axis_label = 'normalized frequency f_n'
fig.yaxis.axis_label = 'S(f_n), M(f_n)'
show(fig)

## Reconstruction of Ideal PAM

### Reconstruction In Frequency Domain
Reconstruction in frequency domain could be achived by setting all values outside the first Nyquist zone to zero.
The first Nyquist zone is given by the number of FFT bins vs. the simulation/PAM sampling frequency `int(f_s_sim/f_s)`

In [76]:
int(f_s_sim/f_s)
N_FFT = len(M)
display(N_FFT)

8945

In [84]:
M_filter = M
M_filter[round(len(M)/2)+int(len(M)/2/int(f_s_sim/f_s)):] = 0.01
M_filter[0:round(len(M)/2)-int(len(M)/2/int(f_s_sim/f_s))] = 0.01

In [87]:
fig = figure(width = 800, height = 300, y_range = (-20,5))
fig.line(x=freqs, y = 10*np.log10(np.abs(M_filter)/np.max(np.abs(M_filter))), color = 'blue', legend = 'M_filt(f)')
#fig.line(x=freqs, y = 10*np.log10(np.abs(M)/np.max(np.abs(M))), color = 'green', legend = 'M(f)')
show(fig)

### Reconstruction in Time Domain

## PAM Signal using Different Pulse Shapes